# Alphabet Soup Charity - Log Binning Attempt

In this notebook, I will attempt to achieve a higher accuracy by applying a log base 10 binning method on the data and re-fitting the model.

## Preprocessing

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
# from google.colab import files
# uploaded = files.upload()

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf
# import io # for Google Colab

#  Import and read the charity_data.csv.
import pandas as pd
application_df = pd.read_csv("./Resources/charity_data.csv") # Local Jupyter Notebook
# application_df = pd.read_csv(io.BytesIO(uploaded['charity_data.csv'])) # Google Colab
application_df.head(3)

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0


In [2]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
df = application_df.drop(['EIN','NAME'], axis=1)
df.head(3)

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0


In [3]:
# Determine the number of unique values in each column.
df.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [4]:
# Look at APPLICATION_TYPE value counts for binning
app_type_value_counts = df['APPLICATION_TYPE'].value_counts()
app_type_value_counts

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T14        3
T25        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [5]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace = []

for x in range(len(app_type_value_counts)):
    if app_type_value_counts.values[x] < 500:
        application_types_to_replace.append(app_type_value_counts.index[x])
        
# Replace in dataframe
for app in application_types_to_replace:
    df['APPLICATION_TYPE'] = df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [6]:
# Look at CLASSIFICATION value counts for binning
classification_val_counts = df['CLASSIFICATION'].value_counts()
classification_val_counts

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C1570        1
C1580        1
C4120        1
C4500        1
C3700        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [7]:
# You may find it helpful to look at CLASSIFICATION value counts >1
for x in range(len(classification_val_counts)):
    if classification_val_counts.values[x] > 1:
        print(f'{classification_val_counts.index[x]}\t{classification_val_counts.values[x]}')

C1000	17326
C2000	6074
C1200	4837
C3000	1918
C2100	1883
C7000	777
C1700	287
C4000	194
C5000	116
C1270	114
C2700	104
C2800	95
C7100	75
C1300	58
C1280	50
C1230	36
C1400	34
C2300	32
C7200	32
C1240	30
C8000	20
C7120	18
C1500	16
C1800	15
C6000	15
C1250	14
C8200	11
C1278	10
C1238	10
C1235	9
C1237	9
C7210	7
C2400	6
C4100	6
C1720	6
C1600	5
C1257	5
C1260	3
C2710	3
C0	3
C3200	2
C1267	2
C1234	2
C1256	2
C1246	2


In [8]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace = []

for x in range(len(classification_val_counts)):
    if classification_val_counts.values[x] < 1500:
        classifications_to_replace.append(classification_val_counts.index[x])

# Replace in dataframe
for cls in classifications_to_replace:
    df['CLASSIFICATION'] = df['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34299 entries, 0 to 34298
Data columns (total 10 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   APPLICATION_TYPE        34299 non-null  object
 1   AFFILIATION             34299 non-null  object
 2   CLASSIFICATION          34299 non-null  object
 3   USE_CASE                34299 non-null  object
 4   ORGANIZATION            34299 non-null  object
 5   STATUS                  34299 non-null  int64 
 6   INCOME_AMT              34299 non-null  object
 7   SPECIAL_CONSIDERATIONS  34299 non-null  object
 8   ASK_AMT                 34299 non-null  int64 
 9   IS_SUCCESSFUL           34299 non-null  int64 
dtypes: int64(3), object(7)
memory usage: 2.6+ MB


In [10]:
df.head(3)

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0


In [11]:
df['ASK_AMT'].max()

8597806340

In [12]:
df['ASK_AMT'].min()

5000

In [13]:
df['ASK_AMT_BINS'] = ''
df

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL,ASK_AMT_BINS
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1,
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1,
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0,
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1,
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1,
...,...,...,...,...,...,...,...,...,...,...,...
34294,T4,Independent,C1000,ProductDev,Association,1,0,N,5000,0,
34295,T4,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0,
34296,T3,CompanySponsored,C2000,Preservation,Association,1,0,N,5000,0,
34297,T5,Independent,C3000,ProductDev,Association,1,0,N,5000,1,


In [23]:
for x in range(len(df.index)):
#     print(df['APPLICATION_TYPE'][x])
    if df['ASK_AMT'][x] > 1_000_000_000:
        df['ASK_AMT_BINS'][x] = 'Greater_than_1_billion'
    elif df['ASK_AMT'][x] > 100_000_000:
        df['ASK_AMT_BINS'][x] = '100_million_to_999_million'
    elif df['ASK_AMT'][x] > 10_000_000:
        df['ASK_AMT_BINS'][x] = '10_million_to_99_million'
    elif df['ASK_AMT'][x] > 1_000_000:
        df['ASK_AMT_BINS'][x] = '1_million_to_9_million'
    elif df['ASK_AMT'][x] > 100_000:
        df['ASK_AMT_BINS'][x] = '100_thousand_to_999_thousand'
    elif df['ASK_AMT'][x] > 10_000:
        df['ASK_AMT_BINS'][x] = '10_thousand_to_99_thousand'
    else:
        df['ASK_AMT_BINS'][x] = 'Less_than_10_thousand'

C:\Users\srobi\anaconda3\envs\PythonAdv\lib\site-packages\ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()
C:\Users\srobi\anaconda3\envs\PythonAdv\lib\site-packages\ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
C:\Users\srobi\anaconda3\envs\PythonAdv\lib\site-packages\ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [26]:
df = df.drop('ASK_AMT',axis=1)
df

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,IS_SUCCESSFUL,ASK_AMT_BINS
0,T10,Independent,C1000,ProductDev,Association,1,0,N,1,Less_than_10_thousand
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,1,100_thousand_to_999_thousand
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,0,Less_than_10_thousand
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,1,Less_than_10_thousand
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,1,100_thousand_to_999_thousand
...,...,...,...,...,...,...,...,...,...,...
34294,T4,Independent,C1000,ProductDev,Association,1,0,N,0,Less_than_10_thousand
34295,T4,CompanySponsored,C3000,ProductDev,Association,1,0,N,0,Less_than_10_thousand
34296,T3,CompanySponsored,C2000,Preservation,Association,1,0,N,0,Less_than_10_thousand
34297,T5,Independent,C3000,ProductDev,Association,1,0,N,1,Less_than_10_thousand


In [27]:
# Convert categorical data to numeric with `pd.get_dummies`
df_dummies = pd.get_dummies(df)
print(df_dummies.columns)
df_dummies

Index(['STATUS', 'IS_SUCCESSFUL', 'APPLICATION_TYPE_Other',
       'APPLICATION_TYPE_T10', 'APPLICATION_TYPE_T19', 'APPLICATION_TYPE_T3',
       'APPLICATION_TYPE_T4', 'APPLICATION_TYPE_T5', 'APPLICATION_TYPE_T6',
       'APPLICATION_TYPE_T7', 'APPLICATION_TYPE_T8',
       'AFFILIATION_CompanySponsored', 'AFFILIATION_Family/Parent',
       'AFFILIATION_Independent', 'AFFILIATION_National', 'AFFILIATION_Other',
       'AFFILIATION_Regional', 'CLASSIFICATION_C1000', 'CLASSIFICATION_C1200',
       'CLASSIFICATION_C2000', 'CLASSIFICATION_C2100', 'CLASSIFICATION_C3000',
       'CLASSIFICATION_Other', 'USE_CASE_CommunityServ', 'USE_CASE_Heathcare',
       'USE_CASE_Other', 'USE_CASE_Preservation', 'USE_CASE_ProductDev',
       'ORGANIZATION_Association', 'ORGANIZATION_Co-operative',
       'ORGANIZATION_Corporation', 'ORGANIZATION_Trust', 'INCOME_AMT_0',
       'INCOME_AMT_1-9999', 'INCOME_AMT_10000-24999',
       'INCOME_AMT_100000-499999', 'INCOME_AMT_10M-50M', 'INCOME_AMT_1M-5M',
       '

,STATUS,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,...,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y,ASK_AMT_BINS_100_million_to_999_million,ASK_AMT_BINS_100_thousand_to_999_thousand,ASK_AMT_BINS_10_million_to_99_million,ASK_AMT_BINS_10_thousand_to_99_thousand,ASK_AMT_BINS_1_million_to_9_million,ASK_AMT_BINS_Greater_than_1_billion,ASK_AMT_BINS_Less_than_10_thousand
0,1,1,0,1,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,1
1,1,1,0,0,0,1,0,0,0,0,...,0,1,0,0,1,0,0,0,0,0
2,1,0,0,0,0,0,0,1,0,0,...,0,1,0,0,0,0,0,0,0,1
3,1,1,0,0,0,1,0,0,0,0,...,0,1,0,0,0,0,0,0,0,1
4,1,1,0,0,0,1,0,0,0,0,...,0,1,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34294,1,0,0,0,0,0,1,0,0,0,...,0,1,0,0,0,0,0,0,0,1
34295,1,0,0,0,0,0,1,0,0,0,...,0,1,0,0,0,0,0,0,0,1
34296,1,0,0,0,0,1,0,0,0,0,...,0,1,0,0,0,0,0,0,0,1
34297,1,1,0,0,0,0,0,1,0,0,...,0,1,0,0,0,0,0,0,0,1


In [ ]:
# Split our preprocessed data into our features and target arrays
y_label = df_dummies['IS_SUCCESSFUL']
X_dummies = df_dummies.drop(['IS_SUCCESSFUL'],axis=1)

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X_dummies, y_label, random_state=1)

In [ ]:
X_train.shape

In [ ]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [ ]:
# Create a method that creates a new Sequential model with hyperparameter options
def create_model(hp):
    nn_model = tf.keras.models.Sequential()

    # Allow kerastuner to decide which activation function to use in hidden layers
    activation = hp.Choice('activation',['relu','tanh','sigmoid'])
    
    # Allow kerastuner to decide number of neurons in first layer
    nn_model.add(tf.keras.layers.Dense(units=hp.Int('first_units',
        min_value=1,
        max_value=30,
        step=2), activation=activation, input_dim=len(X_train_scaled[0])))

    # Allow kerastuner to decide number of hidden layers and neurons in hidden layers
    for i in range(hp.Int('num_layers', 1, 6)):
        nn_model.add(tf.keras.layers.Dense(units=hp.Int('units_' + str(i),
            min_value=1,
            max_value=30,
            step=2),
            activation=activation))
    
    nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

    # Compile the model
    nn_model.compile(loss="binary_crossentropy", optimizer='adam', metrics=["accuracy"])
    
    return nn_model

In [ ]:
# !pip install keras-tuner

In [ ]:
# Import the kerastuner library
import keras_tuner as kt
import os

tuner = kt.Hyperband(
    create_model,
    objective="val_accuracy",
    max_epochs=20,
    directory=os.path.normpath('C:/untitled_project/09'),
    hyperband_iterations=2)

In [ ]:
y_train.shape

In [ ]:
X_train_scaled.shape

In [ ]:
# Run the kerastuner search for best hyperparameters
tuner.search(X_train_scaled,y_train,epochs=20,validation_data=(X_test_scaled,y_test))

In [ ]:
# Get best model hyperparameters
best_hyper = tuner.get_best_hyperparameters(1)[0]
best_hyper.values

In [ ]:
# Evaluate best model against full test data
best_model = tuner.get_best_models(1)[0]

# Export our model to HDF5 file
best_model.save('Sequential008.h5')
model_loss, model_accuracy = best_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")